In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime

%load_ext tensorboard

In [2]:
InputData = np.loadtxt("dataInput2.csv",delimiter=",")
OutputData = np.loadtxt ("dataOutput2.csv", delimiter=",")

In [3]:
InputDataTrain = InputData[0:900]
InputDataTest = InputData[900:1106]

OutputDataTrain = OutputData[0:900]
OutputDataTest = OutputData[900:1106]


InputDataTrain_normalized = tf.keras.utils.normalize( InputDataTrain)
InputDataTest_normalized =  tf.keras.utils.normalize( InputDataTest)
OutputDataTrain_normalized = tf.keras.utils.normalize( OutputDataTrain)
OutputDataTest_normalized =  tf.keras.utils.normalize( OutputDataTest)

In [4]:
train = pd.read_csv("dataInput.csv", names= ["Filter1","Filter2","Filter3","Filter4","Filter5","Filter6","Filter7","Filter8","Filter9","Filter10","Filter11","Filter12","Filter13","Filter14","Filter15","Filter16","Filter17","Filter18","Filter19","Filter20","Filter21","Filter22","Filter23","Filter24","Filter25","Filter26","Filter27","Filter28","Filter29","Filter30","Filter31"])
#train

In [5]:
def create_model() :
    return tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(31, 1)),
      tf.keras.layers.Dense(62,input_dim = 31, activation='relu'),
      tf.keras.layers.Dense(62,input_dim = 62, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(31)
    ])

In [6]:
model = create_model()
model

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=InputDataTrain_normalized, 
          y=OutputDataTrain_normalized, 
          epochs=200, 
          validation_data=(InputDataTest_normalized, OutputDataTest_normalized), 
          callbacks=[tensorboard_callback])

Epoch 1/200

 1/29 [>.............................] - ETA: 0s - loss: 0.0438 - accuracy: 0.0000e+00WARNING:tensorflow:From C:\Users\Robert\anaconda3\envs\ml\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.

 2/29 [=>............................] - ETA: 1s - loss: 0.0421 - accuracy: 0.0000e+00WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0020s vs `on_train_batch_end` time: 0.0765s). Check your callbacks.

29/29 [==============================] - 0s 10ms/step - loss: 0.0355 - accuracy: 0.0278 - val_loss: 0.0314 - val_accuracy: 0.0631
Epoch 2/200

29/29 [==============================] - 0s 2ms/step - loss: 0.0307 - accuracy: 0.0667 - val_loss: 0.0290 - val_accuracy: 0.1068
Epoch 3/200

29/29 [==============================] - 0s 2ms/step - loss: 0

In [ ]:
%tensorboard --logdir logs --host localhost --port 6003

Launching TensorBoard...


In [127]:
print(model.evaluate(InputDataTest_normalized,OutputDataTest_normalized))


7/7 [==============================] - 0s 571us/step - loss: 0.0021 - accuracy: 0.6990
[0.0020722434855997562, 0.6990291476249695]


In [19]:
model.output_shape

(None, 31)

In [32]:
model1 = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[2,4])])
model1.add(tf.keras.layers.Dense(100))
model1.add(tf.keras.layers.Dense(4))
model1.compile(optimizer='sgd', loss='mean_squared_error')


In [96]:
xs = np.array([np.random.rand(2,4) for i in range(1000)])
ys = np.array([xs[i]**2 for i in range(1000)])


TypeError: Cannot interpret 'tf.float32' as a data type

In [97]:
model1.fit(xs, ys, epochs=100, batch_size=1)

Epoch 1/100

1000/1000 [==============================] - 0s 496us/step - loss: 0.0678
Epoch 2/100

1000/1000 [==============================] - 1s 503us/step - loss: 0.0677
Epoch 3/100

1000/1000 [==============================] - 1s 501us/step - loss: 0.0677
Epoch 4/100

1000/1000 [==============================] - 0s 495us/step - loss: 0.0678
Epoch 5/100

1000/1000 [==============================] - 0s 486us/step - loss: 0.0678
Epoch 6/100

1000/1000 [==============================] - 1s 511us/step - loss: 0.0677
Epoch 7/100

1000/1000 [==============================] - 0s 484us/step - loss: 0.0677
Epoch 8/100

1000/1000 [==============================] - 0s 497us/step - loss: 0.0676
Epoch 9/100

1000/1000 [==============================] - 0s 500us/step - loss: 0.0677
Epoch 10/100

1000/1000 [==============================] - 0s 478us/step - loss: 0.0677
Epoch 11/100

1000/1000 [==============================] - 0s 462us/step - loss: 0.0678
Epoch 12/100

1000/1000 [================

In [101]:
p = np.random.rand(1,2,4)*24
p

array([[[11.0894741 ,  4.83823366,  0.47198333,  3.80080368],
        [ 3.19806845, 18.28193717, 16.11627985, 22.07048265]]])

In [68]:
print(model1.predict(p))


[[[0.3463932  0.26626474 0.4240235  0.45840847]
  [0.33611897 0.31240302 0.38057873 0.3982914 ]]]


In [72]:
c = np.array([[1,2,3,4],[5,6,7,8]])
c.shape

(2, 4)

In [71]:
print(model1.predict(c))

[[ 0.68179095 -1.2398953   1.8422542   2.4209013 ]
 [ 1.071494   -2.9899244   3.490117    4.7011495 ]]


In [105]:
d = np.random.uniform(-12,12,size=(1,31))
d

array([[ -2.78735843,  -1.11577067,  -1.81714948,  11.66955254,
         -3.45351712,  -5.05451112,  -2.63361787,   4.18762511,
          5.74083144,   4.52782757,   3.0655622 , -10.93441594,
         -0.03311294,  11.61917753,   7.73278057, -10.16561473,
          3.59488148,  -8.84723986,   6.29434035,   9.20395283,
         -9.20966129,  10.8216834 ,  -5.17979402,   7.61683228,
         -6.29524937, -11.54094152,  -3.5139601 ,   3.75533524,
          9.09961118,   7.87728474,  -4.20901559]])

In [15]:
e = np.ones(31).reshape((1,31)) * 12
e

array([[12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
        12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
        12., 12., 12., 12., 12.]])

In [16]:
model.predict(e)

array([[ 2.4344676 ,  1.2357762 ,  0.45749897,  0.2775921 ,  2.043933  ,
         0.20581484, -0.8364385 ,  1.0970778 , -0.325534  ,  2.5618608 ,
         0.34447086, -0.5555736 , -0.39509127,  1.3997548 ,  1.1635656 ,
         1.1213933 ,  0.9843278 ,  0.556679  , -0.12756859,  0.2619843 ,
         2.1507902 , -1.3481097 ,  1.7620968 ,  2.2843966 ,  0.357185  ,
        -3.0544083 ,  3.0477016 ,  1.2621707 ,  0.48986468,  2.1501207 ,
         1.0494157 ]], dtype=float32)